In [5]:
# file name: R0_ss_sensitive.ipynb

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# robust Pearson (same as I gave earlier)
def pearsonr_np(x, y):
    x = np.asarray(x, float).ravel()
    y = np.asarray(y, float).ravel()
    # drop NaN pairs
    m = ~(np.isnan(x) | np.isnan(y))
    x, y = x[m], y[m]
    if x.size < 2:
        return np.nan, x.size
    x = x - x.mean(); y = y - y.mean()
    sx = np.sqrt(np.dot(x, x)); sy = np.sqrt(np.dot(y, y))
    if sx == 0.0 or sy == 0.0:
        return np.nan, x.size
    r = float(np.dot(x, y) / (sx * sy))
    return max(-1.0, min(1.0, r)), x.size

# --- group by (x1, x2) and correlate x3 vs y1 across the x3 repetitions ---
def corr_one_group(g):
    r, n = pearsonr_np(g["Dimmunity"].values, g["avg_time"].values)
    return pd.Series({"r": r, "n": n})

In [6]:
# --- your CSV ---
df = pd.read_csv("../experimental_data/R0_sigma_Dimmunity_results_v2.csv")   # columns: R0, sigma, Dimmunity

In [28]:
def make_10_pngs_each_10(
    csv_path,
    out_dir="../../figures/R0_vs_avg_time_panels_by_sigma",
    A1_col="R0",
    A2_col="sigma",
    A3_col="Dimmunity",
    B1_col="avg_time",
    dpi=180
):
    os.makedirs(out_dir, exist_ok=True)

    df = pd.read_csv(csv_path)

    # Ensure numeric (important if CSV has 1e+00 etc.)
    for c in [A1_col, A2_col, A3_col, B1_col]:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df = df.dropna(subset=[A1_col, A2_col, A3_col, B1_col])

    A2_vals = np.sort(df[A2_col].unique())
    A3_vals = np.sort(df[A3_col].unique())
    A1_name = A1_col
    A2_name = A2_col
    A3_name = A3_col 
    B1_name = B1_col

    # Expect 10 each, but don't hard-fail if not
    print(f"Unique A2: {len(A2_vals)} values -> {A2_vals}")
    print(f"Unique A3: {len(A3_vals)} values -> {A3_vals}")

    for a2 in A2_vals:
        fig, axes = plt.subplots(2, 5, figsize=(15, 6), sharex=True, sharey=True)
        axes = axes.ravel()

        for k, a3 in enumerate(A3_vals[:10]):  # take first 10 A3s
            ax = axes[k]
            g = df[(df[A2_col] == a2) & (df[A3_col] == a3)].sort_values(A1_col)
            # print(g)

            if g.empty:
                ax.text(0.5, 0.5, "No data", ha="center", va="center", transform=ax.transAxes)
            else:
                ax.scatter(g[A1_col], g[B1_col], s=20)
                ax.plot(g[A1_col], g[B1_col], linewidth=1)  # optional: connect points

            ax.set_title(f"{A3_name}={a3:g}", fontsize=10)

        # If there are fewer than 10 A3 values, hide unused axes
        for k in range(min(len(A3_vals), 10), 10):
            axes[k].axis("off")

        # fig.suptitle(f"A1 vs B1 (fixed A2={a2:g})", fontsize=14)
        fig.suptitle(f"{A1_name} vs {B1_name} (fixed {A2_name}={a2:g})", fontsize=14)

        fig.supxlabel(A1_name)
        fig.supylabel(B1_name)
        fig.tight_layout(rect=[0, 0, 1, 0.97])

        fname = f"{A2_name}_{a2:g}".replace(".", "p")
        fig.savefig(os.path.join(out_dir, fname), dpi=dpi)
        plt.close(fig)

    print(f"Saved 10 panel PNGs to: {out_dir}")
    

Unique A2: 10 values -> [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
Unique A3: 10 values -> [0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5 ]
Saved 10 panel PNGs to: ../../figures/R0_vs_avg_time_panels_by_sigma


In [34]:
def make_10_pngs_each_10(A1_col, B1_col, A2_col, A3_col, out_dir,
                         csv_path="../experimental_data/R0_sigma_Dimmunity_results_v2.csv",
                         dpi=180
                        ):
    os.makedirs(out_dir, exist_ok=True)

    df = pd.read_csv(csv_path)

    # Ensure numeric (important if CSV has 1e+00 etc.)
    for c in [A1_col, A2_col, A3_col, B1_col]:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df = df.dropna(subset=[A1_col, A2_col, A3_col, B1_col])

    A2_vals = np.sort(df[A2_col].unique())
    A3_vals = np.sort(df[A3_col].unique())
    A1_name = A1_col
    A2_name = A2_col
    A3_name = A3_col 
    B1_name = B1_col

    # Expect 10 each, but don't hard-fail if not
    print(f"Unique A2: {A2_name, len(A2_vals)} values -> {A2_vals}")
    print(f"Unique A3: {A3_name, len(A3_vals)} values -> {A3_vals}")

    for a2 in A2_vals:
        fig, axes = plt.subplots(2, 5, figsize=(15, 6), sharex=True, sharey=True)
        axes = axes.ravel()

        for k, a3 in enumerate(A3_vals[:10]):  # take first 10 A3s
            ax = axes[k]
            g = df[(df[A2_col] == a2) & (df[A3_col] == a3)].sort_values(A1_col)
            # print(g)

            if g.empty:
                ax.text(0.5, 0.5, "No data", ha="center", va="center", transform=ax.transAxes)
            else:
                ax.scatter(g[A1_col], g[B1_col], s=20)
                ax.plot(g[A1_col], g[B1_col], linewidth=1)  # optional: connect points

            ax.set_title(f"{A3_name}={a3:g}", fontsize=10)

        # If there are fewer than 10 A3 values, hide unused axes
        for k in range(min(len(A3_vals), 10), 10):
            axes[k].axis("off")

        # fig.suptitle(f"A1 vs B1 (fixed A2={a2:g})", fontsize=14)
        fig.suptitle(f"{A1_name} vs {B1_name} (fixed {A2_name}={a2:g})", fontsize=14)

        fig.supxlabel(A1_name)
        fig.supylabel(B1_name)
        fig.tight_layout(rect=[0, 0, 1, 0.97])

        fname = f"{A2_name}_{a2:g}".replace(".", "p")
        fig.savefig(os.path.join(out_dir, fname), dpi=dpi)
        plt.close(fig)

    print(f"Saved 10 panel PNGs to: {out_dir}")
    

In [35]:
make_10_pngs_each_10(A1_col="R0", B1_col="avg_time", A2_col="sigma", A3_col="Dimmunity", 
                     out_dir="../../figures/R0_vs_avg_time_panels_by_sigma")

Unique A2: ('sigma', 10) values -> [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
Unique A3: ('Dimmunity', 10) values -> [0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5 ]
Saved 10 panel PNGs to: ../../figures/R0_vs_avg_time_panels_by_sigma


In [36]:
make_10_pngs_each_10(A1_col="R0", B1_col="max_time", A2_col="sigma", A3_col="Dimmunity", 
                     out_dir="../../figures/R0_vs_max_time_panels_by_sigma")

Unique A2: ('sigma', 10) values -> [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
Unique A3: ('Dimmunity', 10) values -> [0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5 ]
Saved 10 panel PNGs to: ../../figures/R0_vs_max_time_panels_by_sigma


In [37]:
make_10_pngs_each_10(A1_col="R0", B1_col="num_strains", A2_col="sigma", A3_col="Dimmunity", 
                     out_dir="../../figures/R0_vs_num_strains_panels_by_sigma")

Unique A2: ('sigma', 10) values -> [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
Unique A3: ('Dimmunity', 10) values -> [0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5 ]
Saved 10 panel PNGs to: ../../figures/R0_vs_num_strains_panels_by_sigma


In [38]:
make_10_pngs_each_10(A1_col="R0", B1_col="avg_time_repeat", A2_col="sigma", A3_col="Dimmunity", 
                     out_dir="../../figures/R0_vs_avg_time_repeat_panels_by_sigma")

Unique A2: ('sigma', 10) values -> [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
Unique A3: ('Dimmunity', 10) values -> [0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5 ]
Saved 10 panel PNGs to: ../../figures/R0_vs_avg_time_repeat_panels_by_sigma


In [39]:
make_10_pngs_each_10(A1_col="R0", B1_col="var_time_repeat", A2_col="sigma", A3_col="Dimmunity", 
                     out_dir="../../figures/R0_vs_var_time_repeat_panels_by_sigma")

Unique A2: ('sigma', 10) values -> [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
Unique A3: ('Dimmunity', 10) values -> [0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5 ]
Saved 10 panel PNGs to: ../../figures/R0_vs_var_time_repeat_panels_by_sigma


In [40]:
make_10_pngs_each_10(A1_col="R0", B1_col="avg_prev", A2_col="sigma", A3_col="Dimmunity", 
                     out_dir="../../figures/R0_vs_avg_prev_panels_by_sigma")

Unique A2: ('sigma', 10) values -> [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
Unique A3: ('Dimmunity', 10) values -> [0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5 ]
Saved 10 panel PNGs to: ../../figures/R0_vs_avg_prev_panels_by_sigma


In [41]:
make_10_pngs_each_10(A1_col="R0", B1_col="var_prev", A2_col="sigma", A3_col="Dimmunity", 
                     out_dir="../../figures/R0_vs_var_prev_panels_by_sigma")

Unique A2: ('sigma', 10) values -> [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
Unique A3: ('Dimmunity', 10) values -> [0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5 ]
Saved 10 panel PNGs to: ../../figures/R0_vs_var_prev_panels_by_sigma


In [42]:
make_10_pngs_each_10(A1_col="R0", B1_col="avg_div", A2_col="sigma", A3_col="Dimmunity", 
                     out_dir="../../figures/R0_vs_avg_div_panels_by_sigma")

Unique A2: ('sigma', 10) values -> [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
Unique A3: ('Dimmunity', 10) values -> [0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5 ]
Saved 10 panel PNGs to: ../../figures/R0_vs_avg_div_panels_by_sigma


In [43]:
make_10_pngs_each_10(A1_col="R0", B1_col="var_div", A2_col="sigma", A3_col="Dimmunity", 
                     out_dir="../../figures/R0_vs_var_div_panels_by_sigma")

Unique A2: ('sigma', 10) values -> [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
Unique A3: ('Dimmunity', 10) values -> [0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5 ]
Saved 10 panel PNGs to: ../../figures/R0_vs_var_div_panels_by_sigma


In [44]:
make_10_pngs_each_10(A1_col="R0", B1_col="max_abundance", A2_col="sigma", A3_col="Dimmunity", 
                     out_dir="../../figures/R0_vs_max_abundance_panels_by_sigma")

Unique A2: ('sigma', 10) values -> [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
Unique A3: ('Dimmunity', 10) values -> [0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5 ]
Saved 10 panel PNGs to: ../../figures/R0_vs_max_abundance_panels_by_sigma


In [45]:
make_10_pngs_each_10(A1_col="R0", B1_col="avg_npmi", A2_col="sigma", A3_col="Dimmunity", 
                     out_dir="../../figures/R0_vs_avg_npmi_panels_by_sigma")

Unique A2: ('sigma', 10) values -> [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
Unique A3: ('Dimmunity', 10) values -> [0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5 ]
Saved 10 panel PNGs to: ../../figures/R0_vs_avg_npmi_panels_by_sigma


In [46]:
make_10_pngs_each_10(A1_col="R0", B1_col="div_all_isolates", A2_col="sigma", A3_col="Dimmunity", 
                     out_dir="../../figures/R0_vs_div_all_isolates_panels_by_sigma")

Unique A2: ('sigma', 10) values -> [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
Unique A3: ('Dimmunity', 10) values -> [0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5 ]
Saved 10 panel PNGs to: ../../figures/R0_vs_div_all_isolates_panels_by_sigma


In [48]:
# sigma vs summary statistics 

# 1. sigma vs avg_time (ss1)
print("1. sigma vs avg_time (ss1)")
make_10_pngs_each_10(A1_col="sigma", B1_col="avg_time", A2_col="R0", A3_col="Dimmunity", 
                     out_dir="../../figures/sigma_vs_avg_time_panels_by_sigma")

# 2. sigma vs max_time (ss2)
print("2. sigma vs max_time (ss2)")
make_10_pngs_each_10(A1_col="sigma", B1_col="max_time", A2_col="R0", A3_col="Dimmunity", 
                     out_dir="../../figures/sigma_vs_max_time_panels_by_sigma")

# 3. sigma vs num_strains (ss3)
print("3. sigma vs num_strains (ss3)")
make_10_pngs_each_10(A1_col="sigma", B1_col="num_strains", A2_col="R0", A3_col="Dimmunity", 
                     out_dir="../../figures/sigma_vs_num_strains_panels_by_sigma")

# 4. sigma vs avg_time_repeat (ss4)
print("4. sigma vs avg_time_repeat (ss4)")
make_10_pngs_each_10(A1_col="sigma", B1_col="avg_time_repeat", A2_col="R0", A3_col="Dimmunity", 
                     out_dir="../../figures/sigma_vs_avg_time_repeat_panels_by_sigma")

# 5. sigma vs var_time_repeat (ss5)
print("5. sigma vs var_time_repeat (ss5)")
make_10_pngs_each_10(A1_col="sigma", B1_col="var_time_repeat", A2_col="R0", A3_col="Dimmunity", 
                     out_dir="../../figures/sigma_vs_var_time_repeat_panels_by_sigma")

# 6. sigma vs avg_prev (ss6)
print("6. sigma vs avg_prev (ss6)")
make_10_pngs_each_10(A1_col="sigma", B1_col="avg_prev", A2_col="R0", A3_col="Dimmunity", 
                     out_dir="../../figures/sigma_vs_avg_prev_panels_by_sigma")

# 7. sigma vs var_prev (ss7)
print("7. sigma vs var_prev (ss7)")
make_10_pngs_each_10(A1_col="sigma", B1_col="var_prev", A2_col="R0", A3_col="Dimmunity", 
                     out_dir="../../figures/sigma_vs_var_prev_panels_by_sigma")

# 8. sigma vs avg_div (ss8)
print("8. sigma vs avg_div (ss8)")
make_10_pngs_each_10(A1_col="sigma", B1_col="avg_div", A2_col="R0", A3_col="Dimmunity", 
                     out_dir="../../figures/sigma_vs_avg_div_panels_by_sigma")

# 9. sigma vs var_div (ss9)
print("9. sigma vs var_div (ss9)")
make_10_pngs_each_10(A1_col="sigma", B1_col="var_div", A2_col="R0", A3_col="Dimmunity", 
                     out_dir="../../figures/sigma_vs_var_div_panels_by_sigma")

# 10. sigma vs max_abundance (ss10)
print("10. sigma vs max_abundance (ss10)")
make_10_pngs_each_10(A1_col="sigma", B1_col="max_abundance", A2_col="R0", A3_col="Dimmunity", 
                     out_dir="../../figures/sigma_vs_max_abundance_panels_by_sigma")

# 11. sigma vs avg_npmi (ss11)
print("11. sigma vs avg_npmi (ss11)")
make_10_pngs_each_10(A1_col="sigma", B1_col="avg_npmi", A2_col="R0", A3_col="Dimmunity", 
                     out_dir="../../figures/sigma_vs_avg_npmi_panels_by_sigma")

# 12. sigma vs div_all_isolates (ss12)
print("12. sigma vs div_all_isolates (ss12)")
make_10_pngs_each_10(A1_col="sigma", B1_col="div_all_isolates", A2_col="R0", A3_col="Dimmunity", 
                     out_dir="../../figures/sigma_vs_div_all_isolates_panels_by_sigma")

1. sigma vs avg_time (ss1)
Unique A2: ('R0', 12) values -> [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
Unique A3: ('Dimmunity', 10) values -> [0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5 ]
Saved 10 panel PNGs to: ../../figures/sigma_vs_avg_time_panels_by_sigma
2. sigma vs max_time (ss2)
Unique A2: ('R0', 12) values -> [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
Unique A3: ('Dimmunity', 10) values -> [0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5 ]
Saved 10 panel PNGs to: ../../figures/sigma_vs_max_time_panels_by_sigma
3. sigma vs num_strains (ss3)
Unique A2: ('R0', 12) values -> [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
Unique A3: ('Dimmunity', 10) values -> [0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5 ]
Saved 10 panel PNGs to: ../../figures/sigma_vs_num_strains_panels_by_sigma
4. sigma vs avg_time_repeat (ss4)
Unique A2: ('R0', 12) values -> [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
Unique A3: ('Dimmunity', 10) values -> [0.05 0.1  0.15 0.2  0.

In [49]:
# Dimmunity vs summary statistics 

# 1. Dimmunity vs avg_time (ss1)
print("1. Dimmunity vs avg_time (ss1)")
make_10_pngs_each_10(A1_col="Dimmunity", B1_col="avg_time", A2_col="R0", A3_col="sigma", 
                     out_dir="../../figures/Dimmunity_vs_avg_time_panels_by_sigma")

# 2. Dimmunity vs max_time (ss2)
print("2. Dimmunity vs max_time (ss2)")
make_10_pngs_each_10(A1_col="Dimmunity", B1_col="max_time", A2_col="R0", A3_col="sigma", 
                     out_dir="../../figures/Dimmunity_vs_max_time_panels_by_sigma")

# 3. Dimmunity vs num_strains (ss3)
print("3. Dimmunity vs num_strains (ss3)")
make_10_pngs_each_10(A1_col="Dimmunity", B1_col="num_strains", A2_col="R0", A3_col="sigma", 
                     out_dir="../../figures/Dimmunity_vs_num_strains_panels_by_sigma")

# 4. Dimmunity vs avg_time_repeat (ss4)
print("4. Dimmunity vs avg_time_repeat (ss4)")
make_10_pngs_each_10(A1_col="Dimmunity", B1_col="avg_time_repeat", A2_col="R0", A3_col="sigma", 
                     out_dir="../../figures/Dimmunity_vs_avg_time_repeat_panels_by_sigma")

# 5. Dimmunity vs var_time_repeat (ss5)
print("5. Dimmunity vs var_time_repeat (ss5)")
make_10_pngs_each_10(A1_col="Dimmunity", B1_col="var_time_repeat", A2_col="R0", A3_col="sigma", 
                     out_dir="../../figures/Dimmunity_vs_var_time_repeat_panels_by_sigma")

# 6. Dimmunity vs avg_prev (ss6)
print("6. Dimmunity vs avg_prev (ss6)")
make_10_pngs_each_10(A1_col="Dimmunity", B1_col="avg_prev", A2_col="R0", A3_col="sigma", 
                     out_dir="../../figures/Dimmunity_vs_avg_prev_panels_by_sigma")

# 7. Dimmunity vs var_prev (ss7)
print("7. Dimmunity vs var_prev (ss7)")
make_10_pngs_each_10(A1_col="Dimmunity", B1_col="var_prev", A2_col="R0", A3_col="sigma", 
                     out_dir="../../figures/Dimmunity_vs_var_prev_panels_by_sigma")

# 8. Dimmunity vs avg_div (ss8)
print("8. Dimmunity vs avg_div (ss8)")
make_10_pngs_each_10(A1_col="Dimmunity", B1_col="avg_div", A2_col="R0", A3_col="sigma", 
                     out_dir="../../figures/Dimmunity_vs_avg_div_panels_by_sigma")

# 9. Dimmunity vs var_div (ss9)
print("9. Dimmunity vs var_div (ss9)")
make_10_pngs_each_10(A1_col="Dimmunity", B1_col="var_div", A2_col="R0", A3_col="sigma", 
                     out_dir="../../figures/Dimmunity_vs_var_div_panels_by_sigma")

# 10. Dimmunity vs max_abundance (ss10)
print("10. Dimmunity vs max_abundance (ss10)")
make_10_pngs_each_10(A1_col="Dimmunity", B1_col="max_abundance", A2_col="R0", A3_col="sigma", 
                     out_dir="../../figures/Dimmunity_vs_max_abundance_panels_by_sigma")

# 11. Dimmunity vs avg_npmi (ss11)
print("11. Dimmunity vs avg_npmi (ss11)")
make_10_pngs_each_10(A1_col="Dimmunity", B1_col="avg_npmi", A2_col="R0", A3_col="sigma", 
                     out_dir="../../figures/Dimmunity_vs_avg_npmi_panels_by_sigma")

# 12. Dimmunity vs div_all_isolates (ss12)
print("12. Dimmunity vs div_all_isolates (ss12)")
make_10_pngs_each_10(A1_col="Dimmunity", B1_col="div_all_isolates", A2_col="R0", A3_col="sigma", 
                     out_dir="../../figures/Dimmunity_vs_div_all_isolates_panels_by_sigma")

1. Dimmunity vs avg_time (ss1)
Unique A2: ('R0', 12) values -> [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
Unique A3: ('sigma', 10) values -> [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
Saved 10 panel PNGs to: ../../figures/Dimmunity_vs_avg_time_panels_by_sigma
2. Dimmunity vs max_time (ss2)
Unique A2: ('R0', 12) values -> [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
Unique A3: ('sigma', 10) values -> [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
Saved 10 panel PNGs to: ../../figures/Dimmunity_vs_max_time_panels_by_sigma
3. Dimmunity vs num_strains (ss3)
Unique A2: ('R0', 12) values -> [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
Unique A3: ('sigma', 10) values -> [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
Saved 10 panel PNGs to: ../../figures/Dimmunity_vs_num_strains_panels_by_sigma
4. Dimmunity vs avg_time_repeat (ss4)
Unique A2: ('R0', 12) values -> [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
Unique A3: ('sigma', 10) values -> [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]